In [1]:
from torchvision import models
import torch
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import os
import cv2
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [2]:
alexnet = models.alexnet(pretrained=True)
alexnet.classifier = nn.Sequential(*[alexnet.classifier[i] for i in range(5)])

In [3]:
def extractFeatures(img):
    transform = transforms.Compose([            
      transforms.Resize(256),                   
      transforms.CenterCrop(224),               
      transforms.ToTensor(),                    
      transforms.Normalize(                     
      mean=[0.485, 0.456, 0.406],               
      std=[0.229, 0.224, 0.225]                 
    )])
    
    img = img.convert('RGB')
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    out = alexnet(batch_t)
    assert out.shape == torch.Size([1, 4096])
    output = out.detach().numpy()
    output = np.reshape(output, 4096)
    return output

In [4]:
def prepare_data(path):
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    i=1
    for folder_person in os.listdir(path):
        for folder_gesture in os.listdir(path + folder_person):
            for file in os.listdir(path + folder_person + "/" + folder_gesture)[0:30]:
                image = Image.open(os.path.join(path + folder_person + "/" + folder_gesture, file))
                if(i<5):
                    image_with_features = extractFeatures(image)
                    X_train.append(image_with_features)
                    y_train.append(int(folder_gesture[0:2]))
                    i+=1
                else:  
                    image_with_features = extractFeatures(image)
                    X_test.append(image_with_features)
                    y_test.append(int(folder_gesture[0:2]))
                    i=1
    return X_train, y_train, X_test, y_test

In [5]:
DATASET_PATH = "../dataset/"
X_train, y_train, X_test, y_test = prepare_data(DATASET_PATH)

In [6]:
print("Training images :" , len(X_train))
print("Training labels :" , len(y_train))
print("Testing images :" , len(X_test))
print("Testing labels :" , len(y_test))

Training images : 2400
Training labels : 2400
Testing images : 600
Testing labels : 600


In [7]:
def model_building(model, X_train, X_test, y_train, y_test):
    model.fit(X_train,y_train)
    score = model.score(X_test,y_test)
    return score

In [8]:
models = [(LinearSVC(), "Linear Support Vector"), 
          (RandomForestClassifier(), "Random forest"),
          (KNeighborsClassifier(n_neighbors=4), "KNN with 4 neighbors"),
          (KNeighborsClassifier(n_neighbors=5), "KNN with 5 neighbors"),
          (KNeighborsClassifier(n_neighbors=6), "KNN with 6 neighbors"),
          (KNeighborsClassifier(n_neighbors=7), "KNN with 7 neighbors"),
          (KNeighborsClassifier(n_neighbors=8), "KNN with 8 neighbors"),
          (KNeighborsClassifier(n_neighbors=9), "KNN with 9 neighbors"),
          (GaussianNB(), "Naive Bayes")]

for model in models:    
    score = model_building(model[0], X_train, X_test, y_train, y_test)
    print("{} score: {}".format(model[1],score))

/home/sarthak/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/sarthak/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Linear Support Vector score: 0.95
Random forest score: 0.755
KNN with 4 neighbors score: 0.9866666666666667
KNN with 5 neighbors score: 0.985
KNN with 6 neighbors score: 0.9883333333333333
KNN with 7 neighbors score: 0.9816666666666667
KNN with 8 neighbors score: 0.98
KNN with 9 neighbors score: 0.9766666666666667
Naive Bayes score: 0.8216666666666667
